Lines table must follow this schema:

```
class Lines(models.Model):
    lineid = models.CharField(max_length=10, primary_key=True)
    routes = ArrayField(models.CharField(max_length=10))

    def __str__(self):
        return "Line ID: "+self.lineid
    
    class Meta:
        verbose_name_plural = "Lines"
        indexes = [
            models.Index(fields=['lineid'],)
        ]
```

In [1]:
import pandas as pd

# Prevents tables from being truncated.
pd.set_option('display.max_columns', None)

In [4]:
# Read trips text files into pandas dataframe
df_16 = pd.read_csv("/home/isaac/Data/rt_trips_2016_I_DB.txt", sep=';')
df_17 = pd.read_csv("/home/isaac/Data/rt_trips_2017_I_DB.txt", sep=';')

In [5]:
# Concat dataframes into one

trips = pd.concat([df_16, df_17], join="inner")

In [6]:
# Look at example of routes associated with lineid

trips.loc[trips['lineid'] == '39A'].head(5)

,datasource,dayofservice,tripid,lineid,routeid,direction,plannedtime_arr,plannedtime_dep,actualtime_arr,actualtime_dep,basin,tenderlot,suppressed,justificationid,lastupdate,note
11,DB,09-FEB-16 00:00:00,2818069,39A,39A_41,1,85769,84600,85576.0,84452.0,BasDef,NaN,NaN,NaN,12-APR-16 09:35:20,",2428241,"
53,DB,09-FEB-16 00:00:00,2814946,39A,39A_40,1,52081,46800,52531.0,46832.0,BasDef,NaN,NaN,NaN,12-APR-16 09:35:20,",2426175,"
77,DB,09-FEB-16 00:00:00,2826868,39A,39A_43,2,49850,45000,49988.0,45038.0,BasDef,NaN,NaN,NaN,12-APR-16 09:35:20,",2425929,"
82,DB,09-FEB-16 00:00:00,2815902,39A,39A_43,2,55850,51000,56066.0,51003.0,BasDef,NaN,NaN,NaN,12-APR-16 09:35:20,",2416634,"
167,DB,09-FEB-16 00:00:00,2823891,39A,39A_43,2,73499,69600,73415.0,69598.0,BasDef,NaN,NaN,NaN,12-APR-16 09:35:20,",2419757,"


In [7]:
trips = trips[['lineid', 'routeid']]
trips.head(3)

,lineid,routeid
0,145,145_105
1,9,9_7
2,54A,54A_12


In [8]:
# Group routeids into sets associated with single lineids (sets to avoid duplicate routeid values)

trips = trips.groupby('lineid')['routeid'].apply(set)

In [9]:
trips.head(10)

lineid
1                   {1_38, 1_39, 1_37, 1_40, 1_41}
102                         {102_9, 102_10, 102_8}
104                               {104_15, 104_16}
11                    {11_43, 11_41, 11_40, 11_42}
111    {111_7, 111_3, 111_10, 111_9, 111_8, 111_4}
114                                 {114_6, 114_5}
116                                 {116_3, 116_1}
118                                        {118_3}
120                  {120_8, 120_9, 120_10, 120_7}
122               {122_14, 122_15, 122_17, 122_16}
Name: routeid, dtype: object

In [10]:
# 125 unique lines

trips.shape

(125,)

In [11]:
# Reset index to convert groupby to normally indexed dataframe

trips = pd.DataFrame(trips).reset_index()
trips.head(5)

,lineid,routeid
0,1,"{1_38, 1_39, 1_37, 1_40, 1_41}"
1,102,"{102_9, 102_10, 102_8}"
2,104,"{104_15, 104_16}"
3,11,"{11_43, 11_41, 11_40, 11_42}"
4,111,"{111_7, 111_3, 111_10, 111_9, 111_8, 111_4}"


In [12]:
# Convert sets to lists

trips['routeid'] = trips['routeid'].apply(list)

In [13]:
trips.head(5)

,lineid,routeid
0,1,"[1_38, 1_39, 1_37, 1_40, 1_41]"
1,102,"[102_9, 102_10, 102_8]"
2,104,"[104_15, 104_16]"
3,11,"[11_43, 11_41, 11_40, 11_42]"
4,111,"[111_7, 111_3, 111_10, 111_9, 111_8, 111_4]"


In [14]:
# print one of the routeid arrays to ensure strings

print(trips.loc[trips['lineid'] == '39A']['routeid'].values[0])

['39A_44', '39A_43', '39A_40', '39A_45', '39A_42', '39A_41']


In [15]:
# Rename columns

trips.columns = ['lineid', 'routes']

In [16]:
# Save as CSV file

# trips.to_csv('Lines.csv', sep=';', index=False)

In [19]:
# Create engine to send data to database
# make sure tunnel has been created on port 5432 before running

from sqlalchemy import create_engine

URI="localhost"
PORT="5432"
DB = "jetaDb"
USER = "postgres"
PASSWORD = "00001234"
  
# engine = create_engine("postgresql://{}:{}@{}:{}/{}"
#                        .format(USER, PASSWORD, URI, PORT, DB), echo=True)

# trips.to_sql('main_lines', engine, if_exists='append', index=False)